# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns",None)

# Olist packages
from olist.data import Olist
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
import unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [2]:
reviews = Review().get_training_data()
reviews.head()

👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [3]:
data = Olist().get_data()

In [4]:
reviews_data = data['order_reviews']
reviews_data.head()

🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [5]:
orders = data['orders']
orders.head()

💥 Let's merge these three datasets and apply some operations

In [6]:
# YOUR CODE HERE

🚓 Remove the reviews that were written even before receiving the order.

In [7]:
# YOUR CODE HERE

🚓 Remove the reviews for undelivered orders.

In [8]:
# YOUR CODE HERE

✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.



<details>
    <summary>💡<i>Hint</i></summary>

* Drop rows with either a missing title or a missing comment
* Refer to the documentation [pandas.DataFrame.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) and have a look at the `how` argument...

In [9]:
# YOUR CODE HERE

🕵🏻 Let's focus on some columns of interest:

In [10]:
# Focus on certain columns
columns_of_interest = ['review_id',
                       'length_review',
                       'review_score',
                       'order_id',
                       'product_category_name',
                       'full_review']
df = df[columns_of_interest]

In [11]:
df.head()

## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews.

In [12]:
# YOUR CODE HERE

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [14]:
# YOUR CODE HERE

🕵🏻‍♂️ Let's focus on these reviews...

In [15]:
# YOUR CODE HERE

### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [17]:
# YOUR CODE HERE

### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [18]:
# YOUR CODE HERE

#### Document Mixture (of Topics)

In [19]:
# YOUR CODE HERE

👉 Let's report the most important topic for each review

In [21]:
# YOUR CODE HERE

#### Topic Mixture (of Words)

In [22]:
# YOUR CODE HERE

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [24]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [25]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))
        

🕵🏻‍♂️ Print the topics with their topwords:

In [26]:
# YOUR CODE HERE

🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _producto = product_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [27]:
# YOUR CODE HERE

In [28]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [29]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head(3)

## (3.4) Pipeline Tf-Idf and LDA

In [30]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [31]:
# YOUR CODE HERE

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [32]:
pipeline._final_estimator

In [33]:
pipeline._final_estimator.components_

**Document Mixture** with the Pipeline:

In [34]:
# YOUR CODE HERE

**Topic Mixture** with the Pipeline:

In [35]:
# YOUR CODE HERE

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [37]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'), 
                                                          ('review_score', 'std')], 
                                                    ascending = [False, True])
product_categories

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [38]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [39]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id", 
                        "review_score", 
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [40]:
worst_products_reviews["most_important_topic"].value_counts()

In [41]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

In [42]:
[topic_word_mixture[i] for i in bad_frequency]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [43]:
sellers = Seller().get_training_data()
sellers.columns

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [44]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits", 
    ascending = True).head(10)
worst_sellers

### (5.2) Products sold by the worst sellers

In [45]:
products = Product().get_training_data() [["product_id", "category"]]
products

❓ What are the types of products sold by the worst sellers ❓

In [46]:
sellers_product_category = data["order_items"].merge(products, 
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

In [47]:
sellers_product_category.groupby("seller_id").count()

### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [48]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [49]:
bad_reviews_sellers = df.merge(data["order_items"])
bad_reviews_sellers.head(3)

In [50]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
    return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [51]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))
    

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`